In this notebook we process the glaciers of the Antarctic Peninsula. This includes a large range so we will not cover
the full range.

In [2]:
# Import script to embed geolocation.
from IPython.display import HTML
import datetime

import sys
import os
import shutil
import numpy as np
sys.path.extend(['/Users/gertmulder/software/rippl_main'])

from rippl.orbit_geometry.read_write_shapes import ReadWriteShapes
from rippl.SAR_sensors.sentinel.sentinel_download import DownloadSentinel
from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

glacier_shape = [[164.70703125,-71.93815765811694],
            [148.359375, -69.47296854140573],
            [136.7578125,-67.97463396204758],
            [133.154296875, -71.9925780509325],
            [131.66015625,-79.68718415450823],
            [137.4609375,-82.49482361179572],
            [163.65234374999997,-82.99413186927222],
            [175.712890625,-82.68748032987172],
            [175.931640625,-78.98818668348775],
            [163.037109375,-78.56048828398782],
            [157.060546875,-76.84081641443098],
            [159.873046875,-73.92246884621463],
            [164.70703125,-71.93815765811694]]
study_area = ReadWriteShapes()
study_area(glacier_shape)

geojson = study_area.shape

# Try to do the same by creating a shapefile with QGIS, geojson online or a .kml file in google earth.
# study_area.read_kml(kml_path)
# study_area.read_geo_json(geojson_path)
# study_area.read_shapefile(shapefile_path)

The next step in the processing is selection of the right track, as the Sentinel-1 satellite will cover the area on both
ascending and descending tracks. However, to do so, we will have to define the start and end date of our data search, 
because the satellite is not always acquiring data. 
Next search will give a small oversight of the available tracks during our period of interest. In our case we will 
search one week before and one week after the earthquake.

Watch out with extending the search window, this can result in a lot of images. Note that every downloaded file will take approximately 4GB of disk space!

In [3]:
# First we search using a time window

# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = 'HH'

# First we check using a time window
start_date = datetime.datetime(year=2018, month=10, day=1)
end_date = datetime.datetime(year=2019, month=10, day=1)

#find_track = DownloadSentinel(start_date=start_date, end_date=end_date, shape=study_area.shape, sensor_mode=mode,
#                              polarisation=polarisation)
#find_track.sentinel_search_ASF()
#find_track.summarize_search_results(plot_cartopy=False, buffer=2)

After selection of the right track we can start the actual download of the images. In our case we use track 38.

This will download our data automatically to our radar database. Additionally, it will download the precise orbit files.
These files are created within a few weeks after the data acquisition and define the satellite orbit within a few cm
accuracy. These orbits are necessary to accurately define the positions of the radar pixels on the ground later on
in the processing.

In [ ]:
# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = 'HH'

from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

# Create the list of the 4 different stacks.
track_no = 98
stack_name = 'ice_sheet_Antarctica'
tmp_directory = '/media/gert/SAR_processing/SAR_tmp'
if not os.path.exists(tmp_directory):
    os.mkdir(tmp_directory)
coreg_tmp_directory = '/media/gert/SAR_processing/SAR_tmp'
if not os.path.exists(coreg_tmp_directory):
    os.mkdir(coreg_tmp_directory)
ml_grid_tmp_directory = '/media/gert/SAR_processing/SAR_tmp'
if not os.path.exists(ml_grid_tmp_directory):
    os.mkdir(ml_grid_tmp_directory)
# For every track we have to select a master date. This is based on the search results earlier.
# Choose the date with the lowest coverage to create an image with only the overlapping parts.
master_date = datetime.datetime(year=2018, month=4, day=24)

# Number of processes for parallel processing. Make sure that for every process at least 2GB of RAM is available
no_processes = 6

s1_processing = GeneralPipelines(processes=no_processes)
# s1_processing.download_sentinel_data(start_date=start_date, end_date=end_date, track=track_no,
#                                           polarisation=polarisation, shapefile=study_area.shape, data=True, source='ASF')
# s1_processing.create_sentinel_stack(start_date=start_date, end_date=end_date, master_date=master_date, cores=no_processes,
#                                          track=track_no,stack_name=stack_name, polarisation=polarisation,
#                                          shapefile=study_area.shape, mode=mode, product_type=product_type)

# Finally load the stack itself. If you want to skip the download step later, run this line before other steps!
s1_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)

wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190817T105604_20190817T105631_017624_021288_E607.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190817/S1B_IW_SLC__1SSH_20190817T105604_20190817T105631_017624_021288_E607.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190817T105539_20190817T105606_017624_021288_29BC.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190817/S1B_IW_SLC__1SSH_20190817T105539_20190817T105606_017624_021288_29BC.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190817T105511_20190817T105541_017624_021288_E669.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190817/S1B_IW_SLC_

wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190712T105509_20190712T105539_017099_0202C3_F311.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190712/S1B_IW_SLC__1SSH_20190712T105509_20190712T105539_017099_0202C3_F311.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190630T105651_20190630T105718_016924_01FD9C_5F34.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190630/S1B_IW_SLC__1SSH_20190630T105651_20190630T105718_016924_01FD9C_5F34.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190630T105626_20190630T105653_016924_01FD9C_FBB4.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190630/S1B_IW_SLC_

wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190513T105533_20190513T105601_016224_01E88D_5B25.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190513/S1B_IW_SLC__1SSH_20190513T105533_20190513T105601_016224_01E88D_5B25.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190513T105505_20190513T105535_016224_01E88D_DE02.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190513/S1B_IW_SLC__1SSH_20190513T105505_20190513T105535_016224_01E88D_DE02.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190501T105648_20190501T105715_016049_01E2F4_7839.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190501/S1B_IW_SLC_

wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190407T105557_20190407T105624_015699_01D751_8EB8.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190407/S1B_IW_SLC__1SSH_20190407T105557_20190407T105624_015699_01D751_8EB8.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190407T105531_20190407T105559_015699_01D751_4FE2.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190407/S1B_IW_SLC__1SSH_20190407T105531_20190407T105559_015699_01D751_4FE2.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190407T105504_20190407T105534_015699_01D751_9A1D.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190407/S1B_IW_SLC_

wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SSH_20190308T105613_20190308T105641_026245_02EE90_62C3.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190308/S1A_IW_SLC__1SSH_20190308T105613_20190308T105641_026245_02EE90_62C3.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SSH_20190308T105545_20190308T105615_026245_02EE90_19FA.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190308/S1A_IW_SLC__1SSH_20190308T105545_20190308T105615_026245_02EE90_19FA.zip"
wget -c --http-user=gertmulder --http-password=Radar2016 --no-check-certificate "https://datapool.asf.alaska.edu/SLC/SA/S1B_IW_SLC__1SSH_20190302T105646_20190302T105713_015174_01C617_9CA3.zip" -O "/media/gert/SAR_data/radar_database/Sentinel-1/s1_asc_t098/IW_SLC__1SSH_HH/20190302/S1B_IW_SLC_

To define the location of the radar pixels on the ground we need the terrain elevation. Although it is possible to 
derive terrain elevation from InSAR data, our used Sentinel-1 dataset is not suitable for this purpose. Therefore, we
download data from an external source to create a digital elevation model (DEM). In our case we use TanDEM-X data.

However, to find the elevation of the SAR data grid, we have to resample the data to the radar grid first to make it
usable. This is done in the next steps.

In [ ]:
# Some basic settings for DEM creation.
dem_buffer = 0.1        # Buffer around radar image where DEM data is downloaded
dem_rounding = 0.1      # Rounding of DEM size in degrees
dem_type = 'TanDEM-X'      # DEM type of data we download (SRTM1, SRTM3 and TanDEM-X are supported)

# Define both the coordinate system of the full radar image and imported DEM
s1_processing.create_radar_coordinates()
s1_processing.create_dem_coordinates(dem_type=dem_type)

# Download external DEM
s1_processing.download_external_dem(dem_type=dem_type, buffer=dem_buffer, rounding=dem_rounding, n_processes=no_processes)

Using the obtained elevation model the exact location of the radar pixels in cartesian (X,Y,Z) and geographic (Lat/Lon)
can be derived. This is only done for the master or reference image. This process is referred to as geocoding.

In [ ]:
# Geocoding of image
s1_processing.geocoding()

The information from the geocoding can directly be used to find the location of the master grid pixels in the slave
grid images. This process is called coregistration. Because the orbits are not exactly the same with every satellite 
overpass but differ hundreds to a few thousand meters every overpass, the grids are slightly shifted with respect to 
each other. These shift are referred to as the spatial baseline of the images. To correctly overlay the master and slave
images the software coregisters and resamples to the master grid.

To do so the following steps are done:
1. Coregistration of slave to master image
2. Deramping the doppler effects due to TOPs mode of Sentinel-1 satellite
3. Resampling of slave image
4. Reramping resampled slave image.

Due to the different orbits of the master and slave image, the phase of the radar signal is also shifted. We do not 
know the exact shift of the two image, but using the geometry of the two images we can estimate the shift of the phase
between different pixels. Often this shift is split in two contributions:
1. The flat earth phase. This phase is the shift in the case the earth was a perfect ellipsoid
2. The topographic phase. This is the phase shift due to the topography on the ground.
In our processing these two corrections are done in one go.

In [ ]:
# Next step applies resampling and phase correction in one step.
# Polarisation
s1_processing.geometric_coregistration_resampling(polarisation=polarisation, output_phase_correction=True,
                                                  coreg_tmp_directory=coreg_tmp_directory, baselines=False, height_to_phase=True)
shutil.rmtree(coreg_tmp_directory)
os.mkdir(coreg_tmp_directory)

Now we can create calibrated amplitudes, interferograms and coherences.

In [ ]:
# Load the images in blocks to temporary disk (or not if only coreg data is loaded to temp disk)
temporal_baseline = 60
min_timespan = temporal_baseline * 2
# Every process can only run 1 multilooking job. Therefore, in the case of amplitude calculation the number of processes
# is limited too the number of images loaded.
amp_processing_efficiency = 0.5
effective_timespan = np.maximum(no_processes * 6 * amp_processing_efficiency, min_timespan)

no_days = datetime.timedelta(days=int(effective_timespan / 2))
if no_days < (end_date - start_date):
    step_date = start_date
    step_dates = []
    while step_date < end_date:
        step_dates.append(step_date)
        step_date += no_days
    step_dates.append(end_date)

    start_dates = step_dates[:-2]
    end_dates = step_dates[2:]
else:
    end_dates = [end_date]
    start_dates = [start_date]


if not isinstance(polarisation, list):
    pol = [polarisation]
else:
    pol = polarisation

for start_date, end_date in zip(start_dates, end_dates):
    s1_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)
    # We split the different polarisation to limit the number of files in the temporary folder.
    for p in pol:
        for dist in [50, 100, 200]:
            # The actual creation of the calibrated amplitude images
            s1_processing.create_ml_coordinates(standard_type='oblique_mercator', dx=dist, dy=dist, buffer=0, rounding=0)
            s1_processing.prepare_multilooking_grid(polarisation)
            s1_processing.create_calibrated_amplitude_multilooked(polarisation, coreg_tmp_directory=coreg_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_output_tiffs_amplitude()

            s1_processing.create_ifg_network(temporal_baseline=temporal_baseline)
            s1_processing.create_interferogram_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_coherence_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)

            # Create output geotiffs
            s1_processing.create_output_tiffs_coherence_ifg()

            # Create lat/lon/incidence angle/DEM for multilooked grid.
            s1_processing.create_geometry_mulitlooked(baselines=True, height_to_phase=True)
            s1_processing.create_output_tiffs_geometry()

            # The coreg temp directory will only contain the loaded input lines/pixels to do the multilooking. These
            # files will be called by every process so it can be usefull to load them in memory the whole time.
            # If not given, these files will be loaded in the regular tmp folder.
            if coreg_tmp_directory:
                if os.path.exists(ml_grid_tmp_directory):
                    shutil.rmtree(ml_grid_tmp_directory)
                    os.mkdir(ml_grid_tmp_directory)

        if tmp_directory:
            if os.path.exists(tmp_directory):
                shutil.rmtree(tmp_directory)
                os.mkdir(tmp_directory)